# Anotação de entidades mencionadas: Trafilatura e spaCy

## Trafilatura

Ferramenta desenvolvida para obter dados textuais da web, podendo ser utilizada como programa de linha de comandos ou como módulo Python.

Instalação de Trafilatura utilizando `pip`.

In [ ]:
pip install trafilatura

A opção `-h` fornece ajuda.

In [ ]:
!trafilatura -h

Criamos o ficheiro `list.txt` com a lista de URLs que queremos descarregar em formato de texto para construir o nosso corpus. Isto é apenas uma prova de funcionamento. Em ambientes reais, é aconselhável usar Trafilatura como *web crawler*, pesquisando sites inteiros e seguindo recursivamente links (opções de navegação: `--feed`, `--sitemap`, `--crawl`, etc). Se a lista de sites for grande, é melhor explorar a paralelização de processos (`--parallel`).

In [ ]:
%%bash
echo "https://www.publico.pt/2024/07/08/mundo/noticia/patriotas-europa-novo-grupo-parlamentar-direita-radical-reune-partidos-le-pen-orban-salvini-2096807
https://www.publico.pt/2024/06/26/mundo/noticia/alemanha-exige-reconhecimento-direito-israel-existir-conceder-cidadania-2095333
https://www.publico.pt/2024/06/30/mundo/noticia/viktor-orban-anuncia-criacao-novo-grupo-direita-radical-parlamento-europeu-2095827
https://www.publico.pt/2024/06/22/mundo/noticia/luta-barcelona-alojamento-local-transformada-batalha-juridica-2094894" > list.txt

In [ ]:
!cat list.txt

Iniciamos o programa indicando que não queremos comentários (`--no-comments`) nem tabelas (`--no-tables`).

A lista de URLs a extrair está no ficheiro `list.txt` (opção `-i`) e o resultado será guardado em `textos` (opção `-o`).

In [ ]:
!trafilatura --no-comments --no-tables -i list.txt -o texts

*Verificamos* que existem documentos limpos (sem tags HTML) correspondentes às entradas no ficheiro `list.txt`.

In [ ]:
# Lista de documentos na pasta texts

!ls texts

In [ ]:
# Três primeiras linhas de um dos documentos descarregados

!head -3 texts/PbCd6uixgZ3cGf-e.txt

Os titulos dos documentos (primeira linha) não levam pontuação final. O código a seguir carrega cada ficheiro e, se a primeira liña ao finaliza com pontuação, acrescenta um ponto. Também adiciona um salto de linha mais para separar os parágrafos.

In [ ]:
import glob
import regex as re

# Abre cada um dos ficheiros em texts com extensão .txt
for ifile in glob.glob('texts/*.txt'):
    f = open(ifile, 'r')

    # Carrega em lines todas as linhas do ficheiro
    lines = f.readlines()
    f.close()

    # Abre ficheiro para escrita com o mesmo nome.
    # Este ficheiro levará a pontuação final.
    with open(ifile, 'w') as ofile:
        # Elimina brancos no início e no final da primeira linha
        line = lines[0].strip()

        # Se a primeira linha não finaliza em pontuação, acrescenta um ponto
        if not re.search(r"\p{P}$", lines[0]):
            lines[0] = line + ".\n"

        # Escreve de volta todas as linhas (já com a primeira modificada) para
        # o ficheiro de saída
        # Ao unirmos as linhas con '\n' (salto de linha), no final teremos dous
        # saltos en vez de um
        ofile.write("\n".join(lines))

In [ ]:
# Seis primeiras linhas do mesmo ficheiro, agora modificado

!head -6 texts/PbCd6uixgZ3cGf-e.txt

Máis información:

- https://trafilatura.readthedocs.io/en/latest/tutorials.html

## spaCy

spaCy é um módulo ou biblioteca Python para processamento de linguagem natural e suporte para mais de 64 idiomas. Inclui componentes como o reconhecimento das entidades mencionadas, a anotação morfossintáctica, lematização, análise de dependências sintácticas, segmentação em frases, entre outras.

![spaCy pipeline](https://spacy.io/images/pipeline.svg "spaCy pipeline")

URL: https://spacy.io/

### Instalação de spaCy

In [ ]:
pip install spacy

### Instalação de um modelo

Mediante o `spacy download` descarregamos um modelo para português. Os modelos LG (*large*) apresentam *a priori* melhor desempenho que os modelos SM (*small*) ou MD (*medium*), mas também consomem mais recursos.

In [ ]:
!python -m spacy download pt_core_news_lg
# !python -m spacy download pt_core_news_md
# !python -m spacy download pt_core_news_sm

### Carga do módulo e o modelo

Carregamos o módulo (`import spacy`) e o modelo que queremos utilizar. Neste caso os textos estão em português, pelo que utilizaremos um modelo nesta língua.

Os modelos que apresentam maior desempenho são os baseados em *Transformers* (`trf`), mas são também os que requerem mais recursos computacionais e de armazenamento. Atualmente não há modelos baseados em Transformers para português no spaCy.

Mais modelos en: https://spacy.io/models

In [ ]:
import spacy

nlp = spacy.load("pt_core_news_lg")
# nlp = spacy.load("pt_core_news_md")
# nlp = spacy.load("pt_core_news_sm")

Lançámos o *pipeline* de processamento do spaCy chamando `nlp()` e processando um pequeno texto de exemplo.

In [ ]:
sample = "A Google foi fundada pelos cientistas Larry Page e Sergey Brin quando eram estudantes na Universidade de Stanford, na Califórnia."

doc = nlp(sample)

Uma vez lançado o *pipeline*, em `doc` temos os resultados de todos os seus componentes.

In [ ]:
# Componentes da pipeline

nlp.pipe_names

**Etiquetação morfossintática**



In [ ]:
# PoS tagging (tagger)
for token in doc:
    # Imprime cada palabra, o seu lema e a etiqueta morfossintática
    print(f'{token.text} {token.lemma_} {token.pos_}')

**Análise sintática de dependências**

In [ ]:
# Dependency parsing (parser)
for token in doc:
    # Imprime cada palabra, o seu lema, a etiqueta morfossintática, a dependência sintática e o token de que depende (*head*)
    print(f'{token.text} {token.lemma_} {token.pos_} {token.dep_} (head: {token.head})')

**Reconhecimento de Entidades Mencionadas**

In [ ]:
# Named-Entity Recognition and Classification (NERC)
for ent in doc.ents:
    # Imprime cada entidade e o seu tipo
    print(f'{ent.text} {ent.label_}')

Alguns dos componentes (como *parser* ou *NERC*) incluem um módulo de visualização.

Exemplo de visualização do analisador:

In [ ]:
from spacy import displacy

small_doc = nlp("A Google foi fundada pelos cientistas Larry Page e Sergey Brin.")
displacy.render(small_doc, jupyter=True, style="dep", options={"distance": 150})

Exemplo de visualização do NER:

In [ ]:
from spacy import displacy

options = {"colors": {"DATE": "lightgreen"}}
displacy.render(doc, jupyter=True, style="ent", options=options)

### Processado do corpus

Utilizaremos o spaCy para processar os textos do corpus e extrair as entidades mencionadas.

In [ ]:
import spacy
import glob

model = "pt_core_news_lg"

# carregamos o modelo de língua large para PT e desativamos os módulos que não empregaremos
nlp = spacy.load(model, disable=["tagger", "parser", "attribute_ruler"])

for input_file in glob.glob('texts/*.txt'):
    # Abrimos cada um dos ficheiros em texts
    with open(input_file) as ifile:
        text = ifile.read()
        output_file = input_file[:-3] + 'ner'
        # Abrimos o ficheiro de saída, com o mesmo nome mas com extensão .ner
        with open(output_file, 'w') as ofile:
            # Processamos o texto usando o modelo de spaCy
            doc = nlp(text)
            for token in doc:
                # Produzimos a etiqueta em formato IOB2
                # B = Primeiro token de uma entidade mencionada
                # I = Segundo token (ou posterior) de uma entidade mencionada
                # O = Não é uma entidade mencionada
                iob = f'{token.ent_iob_}' if token.ent_iob_ == 'O' else f'{token.ent_iob_}-{token.ent_type_}'

                # Escrevemos a palavra, o lema, a etiqueta morfossintática e a etiqueta IOB2
                print(f'{token.text} {token.lemma_} {token.pos_} {iob}', file=ofile)

In [ ]:
!ls texts

In [ ]:
!head -60 texts/PbCd6uixgZ3cGf-e.ner

Podemos usar o displaCy para mostrar as entidades detectadas.

In [ ]:
from spacy import displacy

with open("texts/PbCd6uixgZ3cGf-e.txt") as ifile:
    text = ifile.read()
    doc = nlp(text)

options = {"colors": {"DATE": "lightgreen"}}
displacy.render(doc, jupyter=True, style="ent", options=options)